In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys
from collections import defaultdict

from scipy.optimize import curve_fit

In [ ]:
def poly( v, a, k ):
    return a*np.power(v,k)

def poly_derivative ( v, a, k):
    return a*k*np.power(v, k-1)

def ipoly( g, a, ea, k, ek ): 
    
    v0 = np.power(g/a, 1./k)
    
    dvda = -(1./k)*np.power(0.5, 1./k)*np.power(1./a, 1./(k+1))
    dvdk = -(1./k**2)*np.power(0.5, 1./k)*np.power( (1./a)*(np.log(1./a)-np.log(0.5)), 1./k )
    
    ev0 = np.sqrt( (dvda*ea)**2 + (dvdk*ek)**2 )
    
    return v0, ev0

def dofit( data_x, data_y, error_y, g_target=0.45 ):
    """
    Args: 
        g_target: target gain in units of 10^7
    Returns: 
        ..
        ..
    """
    
    btrials=[5,6,7,7.5,8]
    trials=0
    b=btrials[0]
    
    params=[]
    pcov=[]
    
    repeatfit=True
    
    while repeatfit:
        try:
            #print("Try fit using b: %.2f" % b)
            params, pcov = curve_fit(poly, data_x, data_y, sigma=error_y, absolute_sigma=True, p0=[1e-13, b])
            repeatfit=False
        except:
            trials=trials+1
            if trials<len(btrials):
                b=btrials[trials]
                #print("Repeat fit using b: %.2f" % b)
            else: 
                #print("Finished the amount of trials available for fit")
                raise 
        else: 
            repeatfit=False
            
            
    stderr = np.sqrt(np.diag(pcov))
    
    xint = np.linspace( np.min(data_x)-10, np.max(data_x)+10, 1000 )
    yint = poly(xint, params[0], params[1])
    
    vnom, evnom= ipoly( g_target, params[0], stderr[0] , params[1], stderr[1] )
    
    #print(params, np.sqrt(np.diag(pcov)), vnom, evnom)
    
    return params, np.sqrt(np.diag(pcov)), xint, yint, vnom, evnom



def plotone( PMT, v, g, eg ):
    fig, ax = plt.subplots(1,1, figsize=(10, 4.8))

    print( v, g)
    
    ax.set_ylabel("Gain [$10^7$ electrons]")
    ax.set_xlabel("Voltage [V]")
    ax.legend(fontsize=16)
    
    kargs={'marker':'o', 'lw':0.0, 'elinewidth':2.0}

    out=ax.errorbar( x=v, y=g, yerr=eg, label="Channel ID: %d" % PMT, **kargs )
    res = dofit( v, g, eg )

    p=res[0]
    ep=res[1]
    xint=res[2]
    yint=res[3]
    vnom=res[4]
    evnom=res[5] 

    label= "FIT $a \cdot V^k$: \n"
    label+="a: %.2e $\pm$ %3.e\n" % ( p[0], ep[0] ) 
    label+="b: %.2f $\pm$ %.3f\n" % ( p[1], ep[1] )
    label+='V($4.5 \cdot 10^6$)= %d $\pm$ %d' % ( vnom, evnom )

    ax.plot(xint, yint, color=out[0].get_color(), lw=1.0, label=label)

    ax.set_ylabel("Gain [$10^7$ electrons]")
    ax.set_xlabel("Voltage [V]")
    ax.legend(fontsize=16)
    
    fig.tight_layout()

    return fig, p[0], p[1]

In [ ]:
c = 279
v = [ 1690, 1720, 1660, 1640, 1590 ]
g = [ 0.257657, 0.28069, 0.237733, 0.221588, 0.193454 ]
eg = [0.000900, 0.001007, 0.001001, 0.001054, 0.001360 ]
fig, a, b, = plotone(c, v, g, eg) 

plt.grid(alpha=0.5,linestyle="dashed")
plt.savefig("ch{}_calib_curv.png".format(c),dpi=100)
plt.show()